# Prueba de bondad de ajuste

In [9]:
import math
import pandas as pd
import numpy as np
import scipy.stats as st

## Distribucion discreta (Poisson)

Se dice que el numero de defectos en unas tarjetas madre sigue una distribucion Poisson. Se obtiene una muestra aleatoria de *n = 60* tarjetas madre y se observa el numero de defectos.

In [119]:
df = pd.DataFrame({'defectos': [0, 1, 2, 3], 'oi': [32, 15, 9, 4]})
df

,defectos,oi
0,0,32
1,1,15
2,2,9
3,3,4


Como no sabemos el promedio de la distribucion Poisson habra que estimarla a partir de los datos que tenemos. La estimacion del promedio de defectos por tarjeta madre es el promedio muestral, o sea, (32 * 0 + 15 * 1 + 9 * 2 + 4 * 3) / 60 = 0.75.

In [120]:
n = df['oi'].sum()
media = 0
for i, row in df.iterrows():
	media += row['oi'] * row['defectos']
media = media / n
media

0.75

O bien, podemos desagrupar los datos y obtener la media con una funcion

In [121]:
defectos = []
for i in df['defectos']:
	for _ in range(df['oi'][i]):
		defectos.append(i)

media = np.mean(defectos)
media

0.75

A partir de la distribucion Poisson con parametro 0.75, puede sacarse $p_{i}$ que es la probabilidad hipotetica asociada con el *i*-ésimo intervalo de clase. Como cada intervalo de clase corresponde a un numero particular de defectos, puede hallarse $p_{i}$ de la sig. manera:

$p_{1} = P(X = 0) = \frac{e^{-0.75}(0.75)^{0}}{0!} = 0.472$
<br>
$p_{2} = P(X = 1) = \frac{e^{-0.75}(0.75)^{1}}{1!} = 0.354$
<br>
$p_{3} = P(X = 2) = \frac{e^{-0.75}(0.75)^{2}}{2!} = 0.133$
<br>
$p_{4} = P(X \ge 3) = 1 - (p_{1} + p_{2} + p_{3}) = 0.041$

In [122]:
col_p = []
for i, num_def in enumerate(df['defectos']):
    if i < (len(df) - 1):
        p = (np.exp(-media) * (media ** num_def)) / math.factorial(num_def)
    else:
        p = 1 - sum(col_p)
    col_p.append(p)
col_p

[0.4723665527410147,
 0.354274914555761,
 0.13285309295841038,
 0.04050543974481391]

In [123]:
col_p = df.apply(lambda i: st.poisson.pmf(i['defectos'], media), axis = 1)
col_p = col_p.to_list()
col_p = col_p[:-1]
col_p.append(1 - sum(col_p))
col_p

[0.4723665527410147,
 0.3542749145557611,
 0.1328530929584104,
 0.0405054397448138]

In [108]:
df['p'] = col_p
df

,defectos,oi,p
0,0,32,0.472367
1,1,15,0.354275
2,2,9,0.132853
3,3,4,0.040505


Las frecuencias esperadas se sacan multiplicando el tamaño de la muestra *n* por las probabilidades $p_{i}$. O sea, $E_{i} = np_{i}$

In [109]:
df['ei'] = df.apply(lambda i: n * i['p'], axis = 1)
df

,defectos,oi,p,ei
0,0,32,0.472367,28.341993
1,1,15,0.354275,21.256495
2,2,9,0.132853,7.971186
3,3,4,0.040505,2.430326


In [110]:
# ver que registro tiene ei menor que 5
for i in df.index:
    if df['ei'][i] < 5:
        print(i, df['oi'][i], df['p'][i], df['ei'][i])

3 4 0.04050543974481391 2.4303263846888346


Como la frecuencia esperada de la ultima celda es menor que 5, se combinan las dos ultimas celdas.

In [111]:
ultimos_dos = df.tail(2)
suma_ultimos_dos = pd.DataFrame([ultimos_dos.sum()])
suma_ultimos_dos['defectos'] = 2
suma_ultimos_dos['oi'] = int(suma_ultimos_dos['oi'])
suma_ultimos_dos

,defectos,oi,p,ei
0,2,13,0.173359,10.401512


In [112]:
df.drop(df.tail(2).index, inplace = True)
df = pd.concat([df, suma_ultimos_dos], ignore_index = True)
df

,defectos,oi,p,ei
0,0,32,0.472367,28.341993
1,1,15,0.354275,21.256495
2,2,13,0.173359,10.401512


In [113]:
df['x2_calc'] = df.apply(lambda x: (x['oi'] - x['ei']) ** 2 / x['ei'], axis = 1)
df

,defectos,oi,p,ei,x2_calc
0,0,32,0.472367,28.341993,0.472127
1,1,15,0.354275,21.256495,1.841495
2,2,13,0.173359,10.401512,0.649150


In [114]:
x2_calc = df['x2_calc'].sum()
x2_calc

2.9627716023964643

Como la media fue calculada a partir de los datos, la $X^2_{\alpha, k - p - 1}$ va a tener 3 - 1 - 1 = 1 grados de libertad

In [115]:
alfa = 0.05
k = len(df['oi'])  # renglones
p = 1  # parametros
gl = k - p - 1
x2_crit = st.chi2.isf(alfa, gl)
x2_crit

3.8414588206941285

In [88]:
if x2_calc >= x2_crit:
	print('H0 SI se rechaza')
else:
	print('H0 NO se rechaza')

H0 NO se rechaza


In [89]:
st.chi2.sf(abs(x2_calc), gl)

0.0852017780009883